<a href="https://colab.research.google.com/github/young-hwanlee/stand_alone_deep_learning/blob/main/%2312_Run_PyTorch_on_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q torch==1.0.0 torchvision
import torch
print(torch.__version__)

1.12.0+cu113


In [ ]:
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np

50000 10000 10000


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        )

True


Number of 1796010 parameters
Epoch: 0, Time: 37.46 seconds, Train Loss: 1.878177259279334, Val Loss: 1.443014980871466, Test Acc: 60.61%
Epoch: 1, Time: 30.04 seconds, Train Loss: 1.2548147423188094, Val Loss: 1.1081783492353898, Test Acc: 64.02%
